In [33]:
import pandas as pd
import os
import pickle
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
import numpy as np

In [34]:
world_bank_projects = pd.read_pickle('../world_bank_projects/world_bank_projects_json.pkl')
loan_features = pd.read_pickle('agreement_features.pickle')

In [35]:
wb_project_names = world_bank_projects['project_name']
doc_project_names = loan_features['Project Name']

We will use fuzzy string matching to pair project names between the two tables. Let's explore a couple of algorithms. The first one compares substrings.

In [52]:
similarities = np.zeros((len(wb_project_names),len(doc_project_names)))
for i in range(len(wb_project_names)):
    if i % 100 == 0:
        print('computing similarities for world bank project name', i)
    for j in range(len(doc_project_names)):
        similarities[i,j] = fuzz.partial_ratio(wb_project_names.iloc[i],doc_project_names.iloc[j])

computing similarities for world bank project name 0
computing similarities for world bank project name 100
computing similarities for world bank project name 200
computing similarities for world bank project name 300
computing similarities for world bank project name 400
computing similarities for world bank project name 500
computing similarities for world bank project name 600
computing similarities for world bank project name 700
computing similarities for world bank project name 800
computing similarities for world bank project name 900
computing similarities for world bank project name 1000
computing similarities for world bank project name 1100
computing similarities for world bank project name 1200
computing similarities for world bank project name 1300
computing similarities for world bank project name 1400
computing similarities for world bank project name 1500
computing similarities for world bank project name 1600
computing similarities for world bank project name 1700
comp

Let's take a look at what the matches end up being.

In [53]:
matches = similarities.argmax(axis=0)
count = 0
for i in range(len(doc_project_names)):
    sim = similarities[matches[i],i]
    if sim > 95:
        count += 1
        print("doc project:\t", doc_project_names.iloc[i])
        print("best match:\t",wb_project_names.iloc[matches[i]])
        print("score:\t", sim)

nterprises Project
score:	 100.0
doc project:	 Water Supply Project
best match:	 Additional Financing for the Greater Beirut Water Supply Project
score:	 100.0
doc project:	 Second Pollution Abatement Project
best match:	 Second Pollution Abatement Project
score:	 100.0
doc project:	 Second State Statistical System Development Project
best match:	 Second State Statistical System Development Project
score:	 100.0
doc project:	 Structural Adjustment Loan
best match:	 CO: Programmatic Labor Reform and Social Structural Adjustment Loan
score:	 100.0
doc project:	 Transport Project
best match:	 Meghalaya Integrated Transport Project
score:	 100.0
doc project:	 Financial Sector Adjustment Loan
best match:	 Programmatic Financial Sector Adjustment Loan I
score:	 100.0
doc project:	 Programmatic Public Sector Development Policy Loan
best match:	 Development Policy Loan
score:	 100.0
doc project:	 Absheron Rehabilitation Program II:  
Integrated Solid Waste Management Project
best match:	 Integ

Hmm, the substring matching means that some generic project names get matched to something more specific.

In [54]:
count

2388

This is a stricter version, which runs faster and just computes the Levenshtein distance between two strings, dividing by the length.

In [36]:
strict_similarities = np.zeros((len(wb_project_names),len(doc_project_names)))
for i in range(len(wb_project_names)):
    if i % 100 == 0:
        print('computing similarities for world bank project name', i)
    for j in range(len(doc_project_names)):
        strict_similarities[i,j] = fuzz.ratio(wb_project_names.iloc[i],doc_project_names.iloc[j])

computing similarities for world bank project name 0
computing similarities for world bank project name 100
computing similarities for world bank project name 200
computing similarities for world bank project name 300
computing similarities for world bank project name 400
computing similarities for world bank project name 500
computing similarities for world bank project name 600
computing similarities for world bank project name 700
computing similarities for world bank project name 800
computing similarities for world bank project name 900
computing similarities for world bank project name 1000
computing similarities for world bank project name 1100
computing similarities for world bank project name 1200
computing similarities for world bank project name 1300
computing similarities for world bank project name 1400
computing similarities for world bank project name 1500
computing similarities for world bank project name 1600
computing similarities for world bank project name 1700
comp

In [37]:
matches = strict_similarities.argmax(axis=0)
count = 0
for i in range(len(doc_project_names)):
    sim = strict_similarities[matches[i],i]
    if sim > 90:
        count += 1
        if sim < 100:
            print("doc project:\t", doc_project_names.iloc[i])
            print("best match:\t",wb_project_names.iloc[matches[i]])
            print("score:\t", sim)

nnection Project
score:	 93.0
doc project:	 Disaster Risk Management Development Policy Loan
With a Catastrophe Deferred Drawdown Option
best match:	 Disaster Risk Management Development Policy Loan with a Catastrophe Deferred Drawdown Option
score:	 98.0
doc project:	 Cebu Bus Rapid Transit Project
best match:	 Cebu Bus Rapid Transit (BRT) Project
score:	 91.0
doc project:	 Third Development Policy Loan to Foster More Inclusive Growth
best match:	 Philippines Development Policy Loan to Foster More Inclusive Growth
score:	 91.0
doc project:	 Cebu Bus Rapid Transit Project
best match:	 Cebu Bus Rapid Transit (BRT) Project
score:	 91.0
doc project:	 National and Regional Roads Rehabilitation Project
best match:	 National and Regional Roads Rehabilitation
score:	 91.0
doc project:	 Gu izhou Rural Development Project
best match:	 Guizhou Rural Development Project
score:	 99.0
doc project:	 Transmission Efficiency Project
best match:	 Transmission Efficiency Project (TEP)
score:	 91.0
doc p

In [38]:
count

1789

Let's use the strict matching, since we can be more confident that the projects will be correctly matched. Also, some project names are not unique, so we'll leave them out from both directions.

In [39]:
wb_value_counts = wb_project_names.value_counts()
doc_value_counts = doc_project_names.value_counts()
project_ids = []
def get_project_id(k):
    if strict_similarities[matches[k],k] > 90:
        project_name = doc_project_names.iloc[k]
        match_name = wb_project_names.iloc[matches[k]]
        if wb_value_counts[match_name] == 1 and doc_value_counts[project_name] == 1:
            return world_bank_projects.id.iloc[matches[k]]
        else:
            return None
    else:
        return None

loan_features['wb_project_id'] = list(map(get_project_id, range(len(loan_features))))

In [40]:
loan_features['wb_project_id'].describe()

count        1074
unique       1006
top       P166303
freq            4
Name: wb_project_id, dtype: object

It looks like there are still some pairs of loans that map to the same project, even with the strict matching and requiring that they be unique. But they should at least have the same type of project, so the resulting sector should be correct still.

In [41]:
loan_features['wb_project_id'].value_counts()

P166303    4
P105002    3
P115566    2
P049537    2
P068368    2
          ..
P159995    1
P070441    1
P150008    1
P044832    1
P096920    1
Name: wb_project_id, Length: 1006, dtype: int64

In [42]:
loan_features.loc[loan_features['wb_project_id'] == 'P166303']

,id,date,country_code,country_name,named_countries,Project Name,project_desc,amount_standard,currency_standard,closing_month,closing_day,closing_year,loan_length in year,wb_project_id
2622,431661468335398275,2014-07-10,lk,Sri Lanka,"[Sri Lanka, United States]",Disaster Risk Management Development Policy Lo...,SCHEDULE 2\n\nThe following table sets forth t...,102000000.0,us dollar,May,31,2017,3.0,P166303
2661,800601468305081068,2014-10-09,sc,Seychelles,"[India, Seychelles, United States]",Disaster Risk Management Development Policy Lo...,SCHEDULE 2\n\nAmortization Schedule\n\nSubject...,7000000.0,us dollar,September,30,2017,3.0,P166303
2901,681591497463996929,2017-05-12,yf,Serbia,"[Serbia, United States]",Disaster Risk Management Development Policy Lo...,SCHEDULE 2\n\nAmortization Schedule\n\nThe fol...,66100000.0,euro,October,31,2020,3.0,P166303
3044,554641531246086557,2018-07-04,ro,Romania,"[Romania, United States]",Disaster Risk Management Development Policy Lo...,None,400000000.0,euro,December,31,2021,3.0,P166303


In [43]:
loan_features.loc[loan_features['wb_project_id'] =='P105002']

,id,date,country_code,country_name,named_countries,Project Name,project_desc,amount_standard,currency_standard,closing_month,closing_day,closing_year,loan_length in year,wb_project_id
2002,586731468267593603,2008-06-06,id,Indonesia,"[Indonesia, United States]",National Program for Community Empowerment in ...,None,41190000.0,us dollar,,,,NaN,P105002
2195,923601468041066358,2010-04-09,id,Indonesia,"[Indonesia, United States]",Third National Program for Community Empowerme...,SCHEDULE 1 \n\nProject Description \n\nThe obj...,785000000.0,us dollar,December,31,2012,2.0,P105002
2354,926481468035093400,2011-07-27,id,Indonesia,"[Indonesia, United States]",Fourth National Program for Community \nEmpow...,SCHEDULE 1 \n\nProject Description \n\nThe obj...,531190000.0,us dollar,June,30,2014,3.0,P105002


In [44]:
loan_features.loc[loan_features['wb_project_id'] == 'P115566']

,id,date,country_code,country_name,named_countries,Project Name,project_desc,amount_standard,currency_standard,closing_month,closing_day,closing_year,loan_length in year,wb_project_id
1983,293771468033359914,2008-03-28,in,India,"[India, United States]",Fourth Power System Development Project,SCHEDULE 1 \n\nProject Description \n\nThe obj...,6.000000e+08,us dollar,July,31,2013,5.0,P115566
2172,252861468252300804,2009-10-13,in,India,"[India, United States]",Fifth Power System Development Project,SCHEDULE 1 \n\nProject Description \n\nThe obj...,1.000000e+09,us dollar,June,30,2015,6.0,P115566


In [45]:
world_bank_projects.loc['P115566']

id                                                                    P115566
regionname                                                         South Asia
countryname                                               [Republic of India]
lendinginstr                                     Investment Project Financing
projectstatusdisplay                                                   Closed
status                                                                 Closed
project_name                           Fifth Power System Development Project
boardapprovaldate                                   2009-09-22 00:00:00+00:00
ibrdcommamt                                                      1000000000.0
idacommamt                                                                  0
totalamt                                                        1,000,000,000
grantamt                                                                    0
countryshortname                                                

In [46]:
loan_features.loc[loan_features['wb_project_id'] == 'P149528']

,id,date,country_code,country_name,named_countries,Project Name,project_desc,amount_standard,currency_standard,closing_month,closing_day,closing_year,loan_length in year,wb_project_id
2927,171461502376530613,2017-07-07,cn,China,"[China, United States]",Second Gansu Cultural and Natural Heritage Pro...,SCHEDULE 1\n\nProject Description\n\nThe objec...,NaN,,December,31,2022,5.0,P149528


In [47]:
world_bank_projects.loc['P149528']

id                                                                    P149528
regionname                                              East Asia and Pacific
countryname                                      [People's Republic of China]
lendinginstr                                     Investment Project Financing
projectstatusdisplay                                                   Active
status                                                                 Active
project_name                CH-Second Gansu Cultural and Natural Heritage ...
boardapprovaldate                                   2017-02-24 00:00:00+00:00
ibrdcommamt                                                       100000000.0
idacommamt                                                                  0
totalamt                                                          100,000,000
grantamt                                                                    0
countryshortname                                                

They mostly look like the same project with "Second" or "Third" put on.

In [48]:
loan_features = loan_features.merge(world_bank_projects[['id','major_sector_name','sector1_name','sector2_name','sector3_name']], left_on='wb_project_id',right_on='id',how='left')

In [49]:
loan_features.rename({'id_x': 'id'}, axis=1,inplace=True)
loan_features.drop('id_y',axis=1,inplace=True)

In [50]:
loan_features.columns

Index(['id', 'date', 'country_code', 'country_name', 'named_countries',
       'Project Name', 'project_desc', 'amount_standard', 'currency_standard',
       'closing_month', 'closing_day', 'closing_year', 'loan_length in year',
       'wb_project_id', 'major_sector_name', 'sector1_name', 'sector2_name',
       'sector3_name'],
      dtype='object')

In [51]:
loan_features.to_pickle('agreement_features_join_projects.pkl')

In [19]:
world_bank_projects.columns

Index(['id', 'regionname', 'countryname', 'lendinginstr',
       'projectstatusdisplay', 'status', 'project_name', 'boardapprovaldate',
       'ibrdcommamt', 'idacommamt', 'totalamt', 'grantamt', 'countryshortname',
       'projectfinancialtype', 'pdo', 'esrc_ovrl_risk_rate', 'borrower',
       'impagency', 'proj_last_upd_date', 'sector1', 'theme1', 'url',
       'totalcommamt', 'cons_serv_reqd_ind', 'closingdate',
       'envassesmentcategorycode', 'sector2', 'sector3', 'project_abstract',
       'theme2', 'sector1_name', 'sector1_pct', 'sector2_name', 'sector2_pct',
       'sector3_name', 'sector3_pct', 'major_sector_name'],
      dtype='object')

In [32]:
loan_features.dropna(how='any',inplace=False,subset=['project_desc','Project Name','major_sector_name']).describe()

,id,amount_standard,loan_length in year
count,9.070000e+02,7.740000e+02,893.000000
mean,5.337426e+17,1.862315e+08,2.403135
std,2.548703e+17,6.985554e+08,60.910642
min,1.016115e+17,2.000000e+05,-1815.000000
25%,3.182015e+17,3.000000e+07,4.000000
50%,5.163015e+17,8.400000e+07,5.000000
75%,7.490465e+17,1.800000e+08,6.000000
max,9.995615e+17,1.178000e+10,9.000000
